# Datos iniciales

In [ ]:
#librerías
import plotly.graph_objs as go
# https://www.statsmodels.org/dev/tsa.html#exponential-smoothing
from statsmodels.tsa.api import SimpleExpSmoothing # Libreria k usaremos
# lo usaremos para explicar cómo funciona internamente el código
from scipy.optimize import minimize


In [ ]:
#serie que vamos a usar
prod = [77, 92, 32, 37, 69, 87, 73, 41, 88, 107, 82, 82, 97, 78, 84, 67, 105, 141, 159, 130, 127, 86, 60, 104, 14, 98, 88, 105, 51, 72, 116, 59, 106, 51, 60, 93, 75, 62, 124, 64, 86, 77, 57, 73, 72, 115, 85, 66, 78, 86, 90, 77, 48, 27, 41, 42, 43, 89, 45, 29, 67, 61, 67, 69, 133, 97, 67, 81, 68, 56, 44, 73, 53, 34, 67, 35, 57, 65, 86, 27, 53, 24, 85, 103, 72, 90, 82, 57, 104, 70, 99, 66, 25, 52, 81, 96, 100, 83, 102, 63, 77, 89, 108, 93, 123]


In [ ]:
#dibujamos la serie
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1,len(prod))),
                         y=prod,
                         mode='lines+markers',
                         name='Producción'))
fig.update_layout(title='Millones de barriles de petróleo',
                  xaxis_title='Año',
                  yaxis_title='Producción')
fig.show()

# Alisado simple

In [ ]:
x=prod.copy()
alpha=0.2
inicial=77
# Hagamos un alisado exponencial manualmente
xPronostico=[inicial,inicial] #iniciamos con valor "inicial"
for i in range(1,len(x)):
    valor=alpha*x[i]+(1.0-alpha)*xPronostico[-1] #aplicamos la fórmula :: alpha * nueva + (1-alpha) * vieja
    xPronostico.append(valor) # guardamos el valor
#print(xPronostico) #imprimamos todo
#si queremos ver el valor específico:
print("estimador de nivel: ",xPronostico[-1])

estimador de nivel:  94.84926783546041


Valor inicial de forecast: 77

F t + 1 ~ 94.84

# Alisado simple automático

In [ ]:
# Modelo simple con alpha definida por nosotros (0,2)
modelo1 = SimpleExpSmoothing(prod).fit(smoothing_level=0.2, optimized=False)
# Mostramos la información
print(modelo1.summary())
print("valores estimados:",modelo1.forecast(3))

                       SimpleExpSmoothing Model Results                       
Dep. Variable:                  endog   No. Observations:                  105
Model:             SimpleExpSmoothing   SSE                          73657.146
Optimized:                      False   AIC                            692.088
Trend:                           None   BIC                            697.396
Seasonal:                        None   AICC                           692.488
Seasonal Periods:                None   Date:                 Tue, 04 Mar 2025
Box-Cox:                        False   Time:                         22:54:34
Box-Cox Coeff.:                  None                                         
                       coeff                 code              optimized      
------------------------------------------------------------------------------
smoothing_level            0.2000000                alpha                False
initial_level              77.000000                

valor inicial sigue siendo 77


In [ ]:
# Dejamos que el parámetro lo decida el código
modelo2 = SimpleExpSmoothing(prod,initialization_method="estimated").fit()
# podemos decidir qué nivel escoger (generalmente no realizado por el código)
#modelo2 = SimpleExpSmoothing(prod, initialization_method="known", initial_level=77).fit()

print(modelo2.summary())
print("valores futuros:",modelo2.forecast(3))
#print("valores anteriores: ",modelo2.fittedvalues)

                       SimpleExpSmoothing Model Results                       
Dep. Variable:                  endog   No. Observations:                  105
Model:             SimpleExpSmoothing   SSE                          73499.323
Optimized:                       True   AIC                            691.862
Trend:                           None   BIC                            697.170
Seasonal:                        None   AICC                           692.262
Seasonal Periods:                None   Date:                 Tue, 04 Mar 2025
Box-Cox:                        False   Time:                         22:54:36
Box-Cox Coeff.:                  None                                         
                       coeff                 code              optimized      
------------------------------------------------------------------------------
smoothing_level            0.2231772                alpha                 True
initial_level              69.826917                

In [ ]:
# Forecasting 3 periodos
forecast_modelo1 = modelo1.forecast(3)
forecast_modelo2 = modelo2.forecast(3)
# Indices de tiempo para el gráfico
periodo=list(range(1,len(prod)))
periodoFuturo=[len(prod)+1,len(prod)+2,len(prod)+3]

# Ploteemos
fig = go.Figure()

# Serie original
fig.add_trace(go.Scatter(x=periodo, y=prod, mode='lines+markers', line=dict(color='black'), name='Serie original'))
# Valores del fit
fig.add_trace(go.Scatter(x=periodo, y=modelo1.fittedvalues, mode='lines+markers', line=dict(color='red'), name='Modelo 1'))
fig.add_trace(go.Scatter(x=periodo, y=modelo2.fittedvalues, mode='lines+markers', line=dict(color='blue'), name='Modelo 2'))
# Forecast
fig.add_trace(go.Scatter(x=periodoFuturo, y=forecast_modelo1, mode='lines+markers', line=dict(color='red'), name='Forecast modelo 1'))
fig.add_trace(go.Scatter(x=periodoFuturo, y=forecast_modelo2, mode='lines+markers', line=dict(color='blue'), name='Forecast modelo 2'))

fig.show()

# Proceso manual

In [ ]:

# Copiemos los datos para evitar errores
x=prod.copy()

# Definamos la función de error (tenemos los parámetros y los datos)
def mse(params, serie):

    inicial, alpha = params
    xPronostico=[inicial] #iniciamos con valor "inicial"
    for i in range(len(serie)):
        valor=alpha*serie[i]+(1.0-alpha)*xPronostico[-1] #aplicamos la fórmula
        xPronostico.append(valor) # guardamos el valor
    error = 0
    for i in range(len(serie)):
        error += (serie[i]-xPronostico[i])**2
    return error  # Error cuadrático

# valores iniciales
initial_params = [77, 0.15]

# Optimicemos
result = minimize(mse, initial_params, args=(x,), method='L-BFGS-B')

# Imprimamos
if result.success:
    print(f"Parámetros optimización: {result.x}")
else:
    print("Error:", result.message)


Parámetros optimización: [69.82703437  0.22317708]
